# Imports

In [ ]:
import random
import pandas as pd
from numpy.random import default_rng
from datetime import datetime, timedelta
import max_capacities

rng = default_rng(13)

# Global Variables

In [ ]:
REALIZATIONS = 1
NUMBER_OF_AIRPLANES = 5
NUMBER_OF_AIRPORTS = 3

DESTINATIONS = ["Vancouver", "Toronto", "Montreal"]

TORONTO_DIST_OTHER_AIRPORTS = {
    "Vancouver": 3345,
    "Montreal": 537
}

VANCOUVER_DIST_OTHER_AIRPORTS = {
    "Toronto": 3345,
    "Montreal": 3682
}

MONTREAL_DIST_OTHER_AIRPORTS = {
    "Vancouver": 3682,
    "Toronto": 537
}

BRANDS = ["Air Canada", "West Jet", "Porter", "Flair"] 
BRAND_WEIGHTS = [0.5, 0.1, 0.15, 0.25]

AIR_CANADA_MAX_CAPACITIES = max_capacities.AIR_CANADA_MAX_CAPACITIES
AIR_CANADA_WEIGHTS = max_capacities.AIR_CANADA_WEIGHTS

WEST_JET_MAX_CAPACITIES = max_capacities.WEST_JET_MAX_CAPACITIES
WEST_JET_WEIGHTS = max_capacities.WEST_JET_WEIGHTS

PORTER_MAX_CAPACITIES = max_capacities.PORTER_MAX_CAPACITIES
PORTER_WEIGHTS = max_capacities.PORTER_WEIGHTS

FLAIR_MAX_CAPACITIES = max_capacities.FLAIR_MAX_CAPACITIES
FLAIR_WEIGHTS = max_capacities.FLAIR_WEIGHTS


INCREMENT = 15
TIME = 0
END_TIME = 3000

In [ ]:
flight_times_df = pd.read_csv("flight_times.csv")

# Class Definitions

In [ ]:
class Airplane():
    def __init__(self):
        
        # initializing class attributes
        self.destination, self.origin = rng.choice(DESTINATIONS, 2, replace=False)
        random_arrival_time = rng.integers(0,1441)
        self.goal_arrival_time = random_arrival_time
        self.arrival_time = random_arrival_time
        self.brand = rng.choice(BRANDS, 1, replace=True, p=BRAND_WEIGHTS).item()
        self.status = "flying" # Options: unallocated, flying, arriving, departing, blocked, cancelled, stalled, maintaining, landed
        self.needs_maintenance = False
        self.last_maintenance_time = 0
        self.delay = 0

        # Initializing max capacities and maintenance time
        if self.brand == "Air Canada":
            self.max_capacity = rng.choice(AIR_CANADA_MAX_CAPACITIES, 1, replace=True, p=AIR_CANADA_WEIGHTS).item()
            self.maintenance_time = 30
            self.time_between_mandatory_maintenance = 1000
            self.cleaning_time = 15
        elif self.brand == "West Jet":
            self.max_capacity = rng.choice(WEST_JET_MAX_CAPACITIES, 1, replace=True, p=WEST_JET_WEIGHTS).item()
            self.maintenance_time = 30
            self.time_between_mandatory_maintenance = 1000
            self.cleaning_time = 15
        elif self.brand == "Porter":
            self.max_capacity = rng.choice(PORTER_MAX_CAPACITIES, 1, replace=True, p=PORTER_WEIGHTS).item()
            self.maintenance_time = 30
            self.time_between_mandatory_maintenance = 1000
            self.cleaning_time = 15
        elif self.brand == "Flair":
            self.max_capacity = rng.choice(FLAIR_MAX_CAPACITIES, 1, replace=True, p=FLAIR_WEIGHTS).item()
            self.maintenance_time = 30
            self.time_between_mandatory_maintenance = 1000
            self.cleaning_time = 15

        def set_destination(self):
            available_destinations = [d for d in DESTINATIONS if d != self.origin]
            self.destination = rng.choice(available_destinations)

        # Setting other class attributes
        #self.load_percent = load_percent
        #self.goal_departure_time = goal_departure_time
        #self.arrival_time = arrival_time
        #self.departure_time = departure_time

In [ ]:
class Airport():
    def __init__(self, location, distance_to_other_airports):
        
        # initializing class attributes
        self.location = location
        self.airspace_capacity = 5 # Doesn't change
        self.airspace = 3
        self.runway_capacity_large = 2 # Doesn't change
        self.runway_space_large = 1
        self.runway_capacity_small = 2 # doesn't change
        self.runway_space_small = 1
        #self.pdf_delay = pdf_delay
        self.distance_to_other_airports = distance_to_other_airports
        self.congestion = 0.0
        self.buffer_time = 30
        self.redirect_threshold = 0
        #self.operation_information = operation_information

        # ones that I added
        self.delay_end_time = 0

# Initializing Agents

In [ ]:
# Initializing Airplanes
airplanes = []
for i in range(NUMBER_OF_AIRPLANES):
    airplanes.append(Airplane())

In [ ]:
# Initializing Airports
toronto_airport = Airport(location="Toronto", distance_to_other_airports=TORONTO_DIST_OTHER_AIRPORTS)
vancouver_airport = Airport(location="Vancouver", distance_to_other_airports=VANCOUVER_DIST_OTHER_AIRPORTS)
montreal_airport = Airport(location="Montreal", distance_to_other_airports=MONTREAL_DIST_OTHER_AIRPORTS)

airports = [toronto_airport, vancouver_airport, montreal_airport]

# Simulations

In [ ]:
for realization in range(REALIZATIONS):

    while (TIME < END_TIME):

        # ---------- ITERATING THROUGH AIRPORTS -------------#
        for airport in airports:


            # ------- Evaluating Models + Psuedo Agents ------#

            """
            
                Evaluate (Daily Variation Model):
                    Check (passenger trends) contingent on time of day:
                        Update (average passenger count)
                    Check (congestion trends) contingent on time of day:
                        Update (average congestion)

                Evaluate (Passenger Model):
                    Check (passenger count) contingent on average passenger count:
                        Update (airport passenger count)

                Evaluate (Ghost Plane Model):
                    Check (average congestion) contingent on average congestion
                        Update (congestion)

                Evaluate (Delay Pseudo Agent):
                    If current time < airport delay end time:
                        Update (congestion to 100%)
                    else:
                        Check (airport delay [Boolean]):
                            Update (airport delay end time)
                        for airplane departing current airport:
                            Check (airplane delay [Boolean]):
                                Update (airplane departure time)
                                Update (airplane arrival time)
            
            
            """

            # ------- Evaluating Airport Agents ------#

            # Obtaining a list of airplanes where it's destination is the current airport in the loop
            relevant_airplanes_list = [airplane for airplane in airplanes if airplane.destination == airport.location]
            # Updating airspace, runway spaces
            airport.airspace *= (1 - airport.congestion)
            airport.runway_space_large *= (1 - airport.congestion)
            airport.runway_space_small *= (1 - airport.congestion)
            # Updating runway spaces for planes that landed
            for airplane in relevant_airplanes_list:
                if airplane.status == "landed":
                    airport.runway_space_large += 1
                    airport.runway_space_small += 1
                
            # Checking if airplanes status needs to be changed to blocked
            if airport.delay_end_time - TIME > airport.redirect_threshold:
                for airplane in relevant_airplanes_list:
                    airplane.status = "blocked"
            else:
                # ------- Arriving Airplanes --------#
                arriving_airplanes_list = [airplane for airplane in relevant_airplanes_list if airplane.status == "arriving"]
                num_of_arriving_airplanes = len(arriving_airplanes_list)
                if (num_of_arriving_airplanes > airport.airspace) or (num_of_arriving_airplanes > airport.runway_space_large): # TO DO: Should this be done for runway space small as well? can we just make runway space cohesive?
                    pass # TO DO: updating airplanes to landed or stalled 
                else:
                    for arriving_airplanes in arriving_airplanes_list:
                        arriving_airplanes.status = "landed"
                        arriving_airplanes.origin = airport.location
                # TO DO: Update Airspace
                # ----------------------------------#


                # ------- Departing Airplanes ------#
                departing_airplanes_list = [airplane for airplane in airplanes if airplane.origin == airport.location and airplane.status == "departing"]
                num_of_departing_airplanes = len(departing_airplanes_list)
                if (num_of_departing_airplanes > airport.airspace):
                    available_spots = num_of_departing_airplanes - airport.airspace
                    chosen_airplanes_to_depart = rng.choice(departing_airplanes_list, available_spots, replace=False)
                    for airplane in chosen_airplanes_to_depart:
                        airplane.status = "flying"
                        # airplane.set_destination() TO DO: this was in the pseudo code but doesnt make sense
                else:
                    for airplane in departing_airplanes_list:
                        airplane.status = "flying"
                        # airplane.set_destination() TO DO: this was in the pseudo code but doesn't make sense
                # ----------------------------------#

                
                # ------- for airplanes coming to the airport where status is cancelled ------#
                # TO DO
                # ----------------------------------------------------------------------------#

        # ---------- ITERATING THROUGH AIRPLANES -------------#
        for airplane in airplanes:
            
            # ------- Maintenance Model ------#
            if airplane.status == "maintaining":
                airplane.goal_departure_time += airplane.maintenance_time 
                airplane.departure_time += airplane.maintenance_time 
                airplane.last_maintenance_time = TIME
                airplane.status = "landed"

            # ------- Evaluating Airplane Agents------#
            if airplane.arrival_time > airplane.time_between_mandatory_maintenance + airplane.last_maintenance_time:
                airplane.status = "maintaining"

            if airplane.status == "blocked":
                pass # TO DO: handle blocked airplanes
            
            elif airplane.status == "landed" and airplane.departure_time < TIME:
                time_delayed = max(0, airplane.arrival_time - airplane.goal_arrival_time) # ensure delay is >= 0
                airplane.delay += time_delayed # adding delay
                airplane.set_destination() # setting destination

                if airplane.origin == "Vancouver":
                    current_airport = vancouver_airport
                elif airplane.origin == "Toronto":
                    current_airport = toronto_airport
                elif airplane.origin == "Montreal":
                    current_airport = montreal_airport

                avg_time_to_airport = flight_times_df[(flight_times_df['start_destination'] == airplane.origin) & (flight_times_df['end_destination'] == airplane.destination)]['avg_time'].iloc[0]
                current_arrival_time = airplane.arrival_time 
                airplane.arrival_time = current_arrival_time + avg_time_to_airport + 15
                airplane.goal_arrival_time = current_arrival_time + avg_time_to_airport + 15
                airplane.departure_time = current_arrival_time + airplane.cleaning_time + current_airport.buffer_time
                airplane.goal_departure_time = current_arrival_time + airplane.cleaning_time + current_airport.buffer_time
                # TO DO Update load percent

            elif airplane.status == "stalled":

                """					
                Update (arrival time)
                Update (status to flying)
                """
                pass

            # TO DO
            #elif airplane.arrival_time - airplane.goal_arrival_time > airplane.threshold_time: 
            #    airplane.status = "cancelled"
            

            elif (TIME > airplane.departure_time) and (airplane.status == "landed"):
                airplane.status = "departing"
            
            elif (TIME > airplane.arrival_time):
                airplane.status = "arriving"
            
            """
            Evaluate (Delay Cost Model):
                Check (delay)
                Check (cancelled passengers)
                    Update (cost of delay)
            """
        
        TIME += INCREMENT

        